# Step 1: Import Pandas to read the data

In [ ]:
import pandas as pd
data = pd.read_csv('census.csv')
data.head()

In [ ]:
data['income_bracket'].unique()
def lable_fix(lable):
    if lable == '<=50K':
        return 0
    else:
        return 1

In [ ]:
data['income_bracket'] = data['income_bracket'].apply(lable_fix)

# Step 2: Perfrom Train test split

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop('income_bracket',axis=1)
y = data['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Step 3: Import Tensorflow and Create Feature Columns
## Tensorflow offer API called Estimator

In [2]:
import tensorflow as tf
# create a feature columns_ categorical values and create a feature columns_Numberic Values 
# hash buket is nothing but max possible category for unknown column
education      = tf.feature_column.categorical_column_with_hash_bucket("education",hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status",hash_bucket_size=1000)
occupation     = tf.feature_column.categorical_column_with_hash_bucket("occupation",hash_bucket_size=1000)
relationship   = tf.feature_column.categorical_column_with_hash_bucket("relationship",hash_bucket_size=1000)
race           = tf.feature_column.categorical_column_with_hash_bucket("race",hash_bucket_size=1000)
gender         = tf.feature_column.categorical_column_with_vocabulary_list("gender",['Female','Male'])
native_count   = tf.feature_column.categorical_column_with_hash_bucket("workclass",hash_bucket_size=1000)

In [ ]:
age            = tf.feature_column.numeric_column("age")
education_num  = tf.feature_column.numeric_column("education_num")
capital_loss   = tf.feature_column.numeric_column("capital_loss")
capital_gain   = tf.feature_column.numeric_column("capital_gain")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [1]:
# Assign all Feature Columns to one variable

In [ ]:
feat_cols = [age,education,education_num, marital_status,
             occupation, relationship,race, gender, capital_gain,
            capital_loss,hours_per_week,native_count]

# Step 4: Create Input Fucntion

In [ ]:
inp_func = tf.estimator.inputs.pandas_input_fn(X,y,batch_size=100,num_epochs=None,shuffle=True)
# tf.estimator.inputs.pandas_input_fn ---->template for model
# X,y is from train test split function
# batch_size is number of records for each run
# Epocs = number of attempts ro reach accuracy 

# Step 5: Create a Model

In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)
# Pass feature columns to the model

# Step 6: Train the Model

In [ ]:
model.train(input_fn=inp_func,steps=5000)
# Train the model with input function

# Step 7: Predict and Evalute the Model 

In [ ]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)
# x----> X_test because prediction should be on test
# batch size should be equal to len of x_test

In [ ]:
predictions = list(model.predict(input_fn=pred_fn))

# Step 8: ClassificationReport and Accuracy

In [3]:
from sklearn.metrics import classification_report,accuracy_score

In [ ]:
print(classification_report(y_test,predictions))

# y_test ---> because our prediction should be based on Y_test